In [ ]:
struct delta
    n::Int
    q::Int
    avg::Vector{Vector{Float64}}
end

In [ ]:
function delta(n::Int, q::Int) # Initializes correct size delta
    delta(n,q,[zeros(Float64, q*q) for _ in 1:n*n])
end

In [ ]:
function cart2lin(t::Tuple{Int,Int}, dims::Int) # Cartesian to linear indices conversion
    x,y = t[2], t[1]
    @assert 1 ≤ x ≤ dims && 1 ≤ y ≤ dims
    return y + dims*(x-1)
end

In [ ]:
function delta_update!(δ::delta,i::Int,j::Int,a::Int,b::Int) # Updates the delta structure when the condition is verified
    n = δ.n
    q = δ.q
    δ.avg[cart2lin((i,j),n)][cart2lin((a,b),q)] += 1
    nothing
end